# Problem Formulation

In [1]:
from typing import List, Tuple, Any

In [2]:
# Define an abstract base class for State
class State:
    def __init__(self):
        pass
    def is_goal(self) -> bool:
        """
        Check if the current state is a goal state.
        This method should be overridden in subclasses.
        """
        raise NotImplementedError("Subclasses must implement this method.")
    def successors(self) -> List[Tuple["State", int]]:
        """
        Generate the successors of the current state.
        This method should be overridden in subclasses.
        """
        raise NotImplementedError("Subclasses must implement this method.")
    def __repr__(self) -> str:
        return "State"
    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, State):
            return False
        return True
    def __hash__(self) -> int:
        return hash(self.__repr__())

## Water Jug Problem

In [3]:
class WaterJugState(State):
    def __init__(self, small: int, large: int) -> None:
        self.small = small  # 3-liter jug
        self.large = large  # 5-liter jug

    def is_goal(self) -> bool:
        # goal state is to have 4 liters in the large jug
        return self.large == 4
    
    def __repr__(self) -> str:
        return f"({self.small}, {self.large})"
    
    def __eq__(self, other: Any) -> bool:
        # check if the other object is an instance of WaterJugState
        if not isinstance(other, WaterJugState):
            return False
        # check if the small and large jugs are equal
        return self.small == other.small and self.large == other.large
    
    def __hash__(self) -> int:
        # hash function to allow the state to be used in sets and dictionaries
        return hash((self.small, self.large))   
    
    def successors(self) -> List[Tuple["WaterJugState", int]]:
        # generate all possible successor states
        # this function returns a list of tuples
        # each tuple = (a new state, the amount of water moved as a step cost)
        
        successors = []
        
        # Fill the small jug
        if self.small < 3:      
            # if the small jug is not full, fill the small jug
            successors.append((WaterJugState(3, self.large), 3 - self.small))
        
        # Fill the large jug
        if self.large < 5:
            # if the large jug is not full, fill the large jug
            successors.append((WaterJugState(self.small, 5), 5 - self.large))
        
        # Empty the small jug
        if self.small > 0:     
            # if the small jug is not empty, empty the small jug
            successors.append((WaterJugState(0, self.large), self.small))
        
        # Empty the large jug
        if self.large > 0:      
            # if the large jug is not empty, empty the large jug
            successors.append((WaterJugState(self.small, 0), self.large))
        
        # Pour from small to large
        # amount of water that can be poured from small to large jug
        pour_to_large = min(self.small, 5 - self.large)     
        if pour_to_large > 0:   
            # if the large jug can take more water, pour from small to large jug
            successors.append((WaterJugState(self.small - pour_to_large, 
                                             self.large + pour_to_large), 
                                             pour_to_large))
        
        # Pour from large to small
        # amount of water that can be poured from large to small jug
        pour_to_small = min(self.large, 3 - self.small)
        if pour_to_small > 0:   
            # if the small jug can take more water, pour from large to small jug
            successors.append((WaterJugState(self.small + pour_to_small, 
                                             self.large - pour_to_small), 
                                             pour_to_small))
        
        return successors

In [4]:
# Test the WaterJugState class
# Print all successor states and step costs of the initial state (0, 0)
for state, step_cost in WaterJugState(0, 0).successors():   
    print(f"State: {state}, Step Cost: {step_cost}")

State: (3, 0), Step Cost: 3
State: (0, 5), Step Cost: 5


In [5]:
# The given state is: small jug with 3 liters and large jug with 1 liter
given_state = WaterJugState(3, 1)
# Print all successor states and step costs of the given state (3, 1)
for state, step_cost in given_state.successors():
    print(f"State: {state}, Step Cost: {step_cost}")

State: (3, 5), Step Cost: 4
State: (0, 1), Step Cost: 3
State: (3, 0), Step Cost: 1
State: (0, 4), Step Cost: 3


## 8-Puzzle Problem

In [6]:
class EightPuzzleState(State):
    def __init__(self, board: List, blank_tile: Tuple[int, int]) -> None:
        self.board = board 
        self.blank_tile = blank_tile
        self.goal_state = [[1,2,3],[4,5,6],[7,8,0]]

    def is_goal(self) -> bool:
        # check if the current board is the same as the goal state
        return self.board == self.goal_state
    
    def __repr__(self) -> str:
        # string representation of the board
        return '\n'.join([' '.join(map(str, row)) for row in self.board]) 
    
    def __eq__(self, other: Any) -> bool:
        # check if the other object is an instance of EightPuzzleState
        if not isinstance(other, EightPuzzleState):   
            return False
        # check if the board is equal to the other board
        return self.board == other.board        
    
    def __hash__(self) -> int:
        # convert the board to a tuple of tuples for hashing
        return hash(tuple(tuple(row) for row in self.board))  
    
    def is_valid(self, row: int, col: int) -> bool:
        # return True if the row and column are within the bounds of the board
        return 0 <= row < 3 and 0 <= col < 3    
    
    def successors(self) -> List[Tuple["EightPuzzleState", int]]:
        successors = []
        # get the position of the blank tile
        row, col = self.blank_tile
        # possible moves: up, down, left, right
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]     
        for dr, dc in directions:
            # calculate the new position of the blank tile
            new_row, new_col = row + dr, col + dc
            # check if the new position is valid
            if self.is_valid(new_row, new_col):           
                # create a copy of the board
                new_board = [r[:] for r in self.board]      
                # swap the blank tile with the adjacent tile
                new_board[row][col], new_board[new_row][new_col] = \
                    new_board[new_row][new_col], new_board[row][col] 
                # set the new blank tile position to 0
                new_board[new_row][new_col] = 0             
                successors.append((EightPuzzleState(new_board, (new_row, new_col)), 1))
        return successors

### Note:

When we write `new_board = self.board` in Python, we are **not creating a new board**. Instead, board `new_board` and `self.board` refer to the same object in memory. 
Any changes made to `new_board` will also affect `self.board`. This can lead to **unexpected bugs**, especially during search when exploring multiple states.

To prevent this, we need to create a new copy of the board using

`new_board = [r[:] for r in self.board]`

This creates a new list where each row is a copy of the corresponding row in `self.board`.

In [7]:
initial_board = [[7, 2, 4], [5, 0, 6], [8, 3, 1]]  
initial_blank_tile = (1, 1)  
initial_state = EightPuzzleState(initial_board, initial_blank_tile)
print("Initial State:")
print(initial_state)
print()

print("Successors:")
for state, step_cost in initial_state.successors():
    print(f"State:\n{state}, Step Cost: {step_cost}")
    print()

Initial State:
7 2 4
5 0 6
8 3 1

Successors:
State:
7 0 4
5 2 6
8 3 1, Step Cost: 1

State:
7 2 4
5 3 6
8 0 1, Step Cost: 1

State:
7 2 4
0 5 6
8 3 1, Step Cost: 1

State:
7 2 4
5 6 0
8 3 1, Step Cost: 1



## 8-Queen Problem

In [8]:
class EightQueenState(State):
    def __init__(self, queens: List[int]) -> None:
        self.queens = queens            

    def is_goal(self) -> bool:
        return len(self.queens) == 8

    def __repr__(self) -> str:
        # create an 8x8 board filled with '.'
        board = [['.' for _ in range(8)] for _ in range(8)]     
        for col, row in enumerate(self.queens):
            # place the queens on the board, 
            # we neeed to subtract 1 from the row index 
            # to match the 0-based index of the list
            board[row-1][col] = 'Q'     
        return '\n'.join([' '.join(row) for row in board])
    
    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, EightQueenState):
            return False
        return self.queens == other.queens
    
    def __hash__(self) -> int:
        return hash(tuple(self.queens))
    
    def attack(self, row1: int, col1: int, row2: int, col2: int) -> bool:
        return row1 == row2 or col1 == col2 or abs(row1 - row2) == abs(col1 - col2)
    
    def placable(self, new_row: int, new_col: int) -> bool: 
        for col in range(len(self.queens)):                 
            row = self.queens[col]                          
            if self.attack(new_row, new_col, row, col):     
                return False
        return True
    
    def successors(self) -> List[Tuple["EightQueenState", int]]:
        successors = []
        new_col = len(self.queens)      
        for new_row in range(1, 9):     
            if self.placable(new_row, new_col):
                new_queens = self.queens[:]     
                new_queens.append(new_row)      
                successors.append((EightQueenState(new_queens), 1))
        return successors

In [9]:
initial_state = EightQueenState([])  
print("Initial State:")
print(initial_state)
print()
print("Successors:")
for state, step_cost in initial_state.successors():  
    print(f"State:\n{state}, Step Cost: {step_cost}")  
    print()

Initial State:
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .

Successors:
State:
Q . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . ., Step Cost: 1

State:
. . . . . . . .
Q . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . ., Step Cost: 1

State:
. . . . . . . .
. . . . . . . .
Q . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . ., Step Cost: 1

State:
. . . . . . . .
. . . . . . . .
. . . . . . . .
Q . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . ., Step Cost: 1

State:
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
Q . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . ., Step Cost: 1

State:
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
Q . . .

In [10]:
given_state = EightQueenState([2])  
print("Given State:")
print(given_state)  
print()
print("Successors:")
for state, step_cost in given_state.successors():
    print(f"State:\n{state}, Step Cost: {step_cost}")
    print()

Given State:
. . . . . . . .
Q . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .

Successors:
State:
. . . . . . . .
Q . . . . . . .
. . . . . . . .
. Q . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . ., Step Cost: 1

State:
. . . . . . . .
Q . . . . . . .
. . . . . . . .
. . . . . . . .
. Q . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . ., Step Cost: 1

State:
. . . . . . . .
Q . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. Q . . . . . .
. . . . . . . .
. . . . . . . ., Step Cost: 1

State:
. . . . . . . .
Q . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. Q . . . . . .
. . . . . . . ., Step Cost: 1

State:
. . . . . . . .
Q . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. Q . . . . . ., Step Cost: 1



## Romania Route Finding Problem

In [11]:
class RomaniaRouteState:
    def __init__(self, city: str) -> None:
        self.city = city  # current city
        self.routes = {
            "Arad": [("Zerind", 75), ("Sibiu", 140), ("Timisoara", 118)],
            "Bucharest": [("Pitesti", 101), ("Fagaras", 211), ("Giurgiu", 90), ("Urziceni", 85)],
            "Craiova": [("Drobeta", 120), ("Pitesti", 138), ("Rimnicu Vilcea", 146)],
            "Drobeta": [("Mehadia", 75), ("Craiova", 120)],
            "Eforie": [("Hirsova", 86)],
            "Fagaras": [("Sibiu", 99), ("Bucharest", 211)],
            "Giurgiu": [("Bucharest", 90)],
            "Hirsova": [("Urziceni", 98), ("Eforie", 86)],
            "Iasi": [("Neamt", 87), ("Vaslui", 92)],
            "Lugoj": [("Timisoara", 111), ("Mehadia", 70)],
            "Mehadia": [("Lugoj", 70), ("Drobeta", 75)],
            "Neamt": [("Iasi", 87)],
            "Oradea": [("Sibiu", 151), ("Zerind", 71)],
            "Pitesti": [("Rimnicu Vilcea", 97), ("Craiova", 138), ("Bucharest", 101)],
            "Rimnicu Vilcea": [("Sibiu", 80), ("Pitesti", 97), ("Craiova", 146)],
            "Sibiu": [("Fagaras", 99), ("Rimnicu Vilcea", 80), ("Arad", 140), ("Oradea", 151)],
            "Timisoara": [("Lugoj", 111), ("Arad", 118)],
            "Urziceni": [("Bucharest", 85), ("Hirsova", 98)],
            "Zerind": [("Oradea", 71), ("Arad", 75)]
        }

    def is_goal(self) -> bool:
        return self.city == "Bucharest"\
    
    def __repr__(self) -> str:
        return f"{self.city}"
    
    def __eq__(self, other) -> bool:
        if not isinstance(other, RomaniaRouteState):
            return False
        return self.city == other.city
    
    def __hash__(self) -> int:
        return hash(self.city)
    
    def successors(self) -> List[Tuple["RomaniaRouteState", int]]:
        successors = []
        for city, distance in self.routes[self.city]:
            successors.append((RomaniaRouteState(city), distance))
        return successors

In [12]:
initial_state = RomaniaRouteState("Arad")
print("Initial State:")
print(initial_state)
print()
print("Successors:")
for state, step_cost in initial_state.successors():
    print(f"State: {state}, Step Cost: {step_cost}")

Initial State:
Arad

Successors:
State: Zerind, Step Cost: 75
State: Sibiu, Step Cost: 140
State: Timisoara, Step Cost: 118
